In [603]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [604]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [605]:
import numpy as np
import math

In [606]:
def bck_power_iteration(A, num_iters):
    
    eigenvec = np.random.rand(A.shape[1])

    for i in range(num_iters):
        # calculate the Ab
        b = np.dot(A, eigenvec)

        # calculate the norm
        eigenval = np.linalg.norm(b)

        # normalize the eigenvector
        eigenvec = b / eigenval

    return eigenval, eigenvec

In [607]:
def power_iteration(A, num_iters):
    
    eigenvec = np.random.rand(A.shape[1])
    
    # class enters code here
    #
    
    
    #
    
    return eigenval, eigenvec

In [608]:
def earthquake(k1, k2, k3, m1, m2, m3):
        
    # define the building system matrix
    A = np.array([[(k1+k2)/m1, -k2/m1,      0],
                  [ -k2/m2,    (k2+k3)/m2,  -k3/m2],
                  [ 0,         -k3/m3,      k3/m3]])
    
    return A

In [609]:
def get_eig_scatter(eigenvals, eigenvecs, mode_num, do_power=False):
    
    # floor height for plotting purposes
    h = 10.0
    
    # normalize eigenvectors for plotting purposes
    # such that component for 3rd floor is 1
    num_floors = eigenvecs.shape[0]
    if do_power:
        for i in range(num_floors):
            eigenvecs[i] /= eigenvecs[-1]
    else:
        for i in range(num_floors):
            eigenvecs[:,i] /= eigenvecs[-1,i]
    
    x_vals = np.empty(num_floors + 1)
    x_vals[0] = 0
    if do_power:
        x_vals[1:] = eigenvecs[:]
    else:
        x_vals[1:] = eigenvecs[:, (num_floors-1) - mode_num]
    y_vals = np.array([0, 1*h, 2*h, 3*h], dtype='float')
    plot_name = "deformation mode #" + str(mode_num)
    
    return go.Scatter(name=plot_name, x=x_vals, y=y_vals)
    

In [610]:
def plot_modes(mode_num, num_iters, k1, k2, k3, m1, m2, m3, power):

    A = earthquake(k1, k2, k3, m1, m2, m3)
  
    # use numpy to solve eigenvalues and eigenvectors
    eigenvals, eigenvecs = np.linalg.eig(A)
    
    omega_n = np.sqrt(eigenvals) / (2.0 * math.pi)
        
    num_floors = eigenvecs.shape[0]
    print("eigenvalue = \n", eigenvals[(num_floors-1) - mode_num])
    print("eigenvectors = \n", eigenvecs[:, (num_floors-1) - mode_num])
    print("Resonant frequency all modes = \n", omega_n)

    evec_vals = get_eig_scatter(eigenvals, eigenvecs, mode_num)
    
    # get the largest eigenvalue-eigenvector pair from our power_iteration function
    if mode_num == 2:
        power_eigenval, power_eigenvec = bck_power_iteration(A, num_iters)
    elif mode_num == 0:
        power_eigenval, power_eigenvec = bck_power_iteration(np.linalg.inv(A), num_iters)
        
    if mode_num in [0,2] and power==True:
        print("Power eigenvalues = \n", power_eigenval)
        print("Power eigenvectors = \n", power_eigenvec)
        power_evec_vals = get_eig_scatter(power_eigenval, power_eigenvec, mode_num, do_power=True)
        iplot([evec_vals, power_evec_vals])
    else:
        iplot([evec_vals])

In [611]:
w = interact(plot_modes, mode_num=[0, 1, 2], num_iters=(1,20,1), power=[True,False],
             k1=(1e6, 10e6, 1e6), k2=(1e6, 10e6, 1e6), k3=(1e6, 10e6, 1e6), 
             m1=(5e3,15e3,1e3), m2=(5e3,15e3,1e3), m3=(5e3,15e3,1e3))

interactive(children=(Dropdown(description='mode_num', options=(0, 1, 2), value=0), IntSlider(value=10, descri…